https://www.upwork.com/jobs/Data-Scraping-Building-Permits-Website_~016840721e827a5897/
    

Go to this website (https://fasttrack.ocfl.net/OnlineServices/Permit_Building.aspx) and start with the date 01/01/2020 and do a search.  Sort the building permits by "work".  (See screenshots) Enter into every building permit with the "work" type being "New Construction".  Enter into each permit with this type and add the information into the according excel spreadsheet column.  (See screenshot and corresponding excel file). I have already done one as an example.  Repeat this process for everyday from 01/01/2020 to 06/30/2020. Report progress after three hours of work.

In [1]:
from bs4 import BeautifulSoup 
import requests
import selenium
from selenium import webdriver
from time import sleep
import pandas as pd

In [11]:
browser = webdriver.Chrome(r'C:\Users\38099\Downloads\chromedriver_win32_(1)\chromedriver.exe')
browser.get('https://fasttrack.ocfl.net/OnlineServices/Permit_Building.aspx')
browser.maximize_window()

In [12]:
def work_100():
    browser.find_element_by_xpath('//th[contains(text(), "WORK")]').click()
    browser.find_element_by_name('detail_list_resp_3_length').click()
    browser.find_element_by_xpath('//option[contains(@value, "100")]').click()

In [13]:
def parse_table():
    permit = browser.find_element_by_xpath('//td[contains(@data-title, "PERMIT#")]').text.split('\n')[0]
    apply_date = browser.find_element_by_xpath('//td[contains(@data-title, "APPLY DATE")]').text
    name = browser.find_element_by_xpath('//td[contains(@data-title, "NAME")]').text
    status = browser.find_element_by_xpath('//td[contains(@data-title, "STATUS")]').text
    issue_date = browser.find_element_by_xpath('//td[contains(@data-title, "ISSUE DATE")]').text
    expire_date = browser.find_element_by_xpath('//td[contains(@data-title, "EXPIRE DATE")]').text

    type = browser.find_element_by_xpath('//td[contains(@data-title, "TYPE")]').text
    sub_type = browser.find_element_by_xpath('//td[contains(@data-title, "SUB TYPE")]').text
    work_type = browser.find_element_by_xpath('//td[contains(@data-title, "WORK TYPE")]').text

    address = browser.find_element_by_xpath('//td[contains(@data-title, "ADDRESS")]').text
    parcel = browser.find_element_by_xpath('//td[contains(@data-title, "PARCEL")]').text
    description = browser.find_element_by_xpath('//td[contains(@data-title, "DESCRIPTION")]').text

    page_df = pd.DataFrame({'permit#' : post, 'permit' : permit, 'apply_date' : apply_date, 'name' : name,
                      'status' : status, 'issue_date' : issue_date, 'expire_date': expire_date,
                      'type' : type, 'sub_type' : sub_type, 'work_type' : work_type,
                      'address' : address, 'parcel' : parcel, 'description' : description}, index = [0])    
    return page_df

In [23]:
href_arr = []
work_100()
for i in range(1,5):
    table = browser.find_element_by_class_name('display.responsive.table-hover.stripe.dataTable.no-footer')
    for row in table.find_elements_by_xpath('//tbody/tr'):
        work_type = row.find_element_by_css_selector('td.dataTitleDetailsSearch.sorting_1')
        if work_type.text == 'New Construction':
            post_name = row.find_element_by_css_selector('td.dataTitleDetailsSearch').text
            page_name = browser.find_element_by_class_name('paginate_button.current').text
            href_arr.append([page_name, post_name])
    browser.find_element_by_class_name('paginate_button.next').click()

len(href_arr)

In [25]:
data_df = pd.DataFrame(columns = ('permit#' , 'permit', 'apply_date', 'name', 'status' , 'issue_date' , 'expire_date',
                      'type' , 'sub_type' , 'work_type' ,'address', 'parcel', 'description'))    
for i in range(0,134):
    post = href_arr[i][1]
    page = href_arr[i][0]
    print('post: ', post, '||||| page:', page)
    work_100()
    sleep(10)
    browser.find_element_by_xpath('//div[@class = "dataTables_paginate paging_simple_numbers"]/span/a['+ str(page)+']').click()
    sleep(2)
    browser.find_element_by_xpath('//*[contains(text(), "' + post+ '")]').click()
    sleep(2)
    table_info = parse_table()
    data_df = data_df.append(table_info, ignore_index = True)
    sleep(5)
    browser.find_element_by_id('ContentPlaceHolder1_uc_printback_btnBack').click()
    sleep(8)

In [26]:
data_df.head()

,permit#,permit,apply_date,name,status,issue_date,expire_date,type,sub_type,work_type,address,parcel,description
0,E20010714,E20010714,07/23/20,6232 Aralia Ivy Lane,Issued,07/23/20,01/23/21,Electrical Permit,Commercial,New Construction,6232 Aralia Ivy Ln Winter Garden FL 34787,20-23-27-2711-02-450,all electrical work for a new townhome located...
1,X20001144,X20001144,07/22/20,2071 Bellamere Court,Issued,07/23/20,01/23/21,Gas Permit,Residential,New Construction,2071 Bellamere Ct Windermere FL 34786,05-23-28-0610-00-090,"INSTALLING GAS LINE TO 2 WATER HEATERS, 1 RANG..."
2,B20010362,B20010362,06/10/20,E-MOSS PARK PRESERVE PHASE 2,Issued,07/23/20,01/23/21,Residential Permit,Single Family,New Construction,9422 Juniper Moss Cir Orlando FL 32832,09-24-31-5147-00-360,SFR NEW CONST
3,H20007932,H20007932,07/22/20,15407 Gilligan Court,Issued,07/23/20,01/23/21,Mechanical Permit,Residential,New Construction,15407 Gilligan Ct Winter Garden FL 34787,20-24-27-7826-00-820,Install new 4.0-ton HVAC system with duct work
4,H20007931,H20007931,07/22/20,15040 Mayberry Drive,Issued,07/23/20,01/23/21,Mechanical Permit,Residential,New Construction,15040 Mayberry Dr Winter Garden FL 34787,20-24-27-7826-00-660,Install new 4.0-ton HVAC system with duct work
